## For clustering the eirene outputs dev

In [1]:
using Pkg
using Statistics
using Plots
using LinearAlgebra
using Distances
using Eirene
using StatsBase
using Random
using Distributions
using JLD
using MAT
Pkg.add("ColorSchemes")
using ColorSchemes
include("helper_functions.jl")



function bettiBarFromBarcode(barcode,nSteps)
    
    bettiBar = 0
    
    for bar in eachrow(barcode)
        
        bar_birth = Int(bar[1])
        bar_death = Int(bar[2])
        
        lifetime = bar_death - bar_birth
        
        bettiBar = bettiBar + lifetime
    end
    
    return bettiBar
end


function muBarFromBarcode(barcode,nSteps)
    
    muBar = 0
    
    for bar in eachrow(barcode)
        
        bar_birth = Int(bar[1])
        bar_death = Int(bar[2])
        
        lifetime_scaled = bar_birth*(bar_death - bar_birth)
        
        muBar = muBar + lifetime_scaled
    end
    
    return muBar
end
    

function nuBarFromBarcode(barcode,nSteps)
    
    # Assuming nSteps is the end of the filtration.
    
    nuBar = 0
    
    for bar in eachrow(barcode)
        
        bar_birth = Int(bar[1])
        bar_death = Int(bar[2])
        
        lifetime_scaled = (nSteps - bar_death)*(bar_death - bar_birth)
        
        nuBar = nuBar + lifetime_scaled
    end
    
    return nuBar
end


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-7468476903611371082\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-3882521188107977874\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-7468476903611371082\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

  Updating registry at `/opt/julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `/opt/julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]


nuBarFromBarcode (generic function with 1 method)

In [2]:
# Loop through data with specific output ending and load them
thresh = "005"
rho = 0.05
date_string = "082420"
myreaddir = "/home/jovyan/processed_data/70nodes/thresh_$(thresh)"
half_flag = "both"  #"first" or "second" or "both"


"both"

In [3]:
eirene_output_files = filter(x->occursin("eirene_output",x), readdir(myreaddir))

# Keep only the files with the appropriate number of nodes (700 = 70 nodes)
filter!(x->occursin("700",x), eirene_output_files)

10-element Array{String,1}:
 "DP_500_005_700_30_eirene_output.jld"                       
 "IID_500_005_700_eirene_output.jld"                         
 "RG_500_005_700_30_eirene_output.jld"                       
 "RL_500_005_700_eirene_output.jld"                          
 "assoc_700_100_100_20_20_09_05_005_eirene_output.jld"       
 "coreperiph_700_100_100_20_20_09_05_005_eirene_output.jld"  
 "cosineGeometric_500_005_700_30_eirene_output.jld"          
 "disassort_700_100_100_20_20_09_05_005_eirene_output.jld"   
 "discreteUniformConf_500_005_700_00_10000_eirene_output.jld"
 "geometricConf_500_005_700_001_1000_eirene_output.jld"      

In [4]:
# Load in the data and calculate betti curves. Then save for jason

const NREPS = 50
const NNODES = 70
const MAXDIM = 3

nModels = length(eirene_output_files)
model_names = [split(eirene_output_files[i],"_")[1] for i=1:nModels]

nEdges = binomial(NNODES, 2)
betti_curve_array_all = zeros(NREPS,nEdges,MAXDIM,nModels)
bettiBar_all = zeros(NREPS,MAXDIM,nModels)
muBar_all = zeros(NREPS,MAXDIM,nModels)
nuBar_all = zeros(NREPS,MAXDIM,nModels)
betti_curve_i = []
for (i,file) in enumerate(eirene_output_files)
    
    graph_output_dict = load("$(myreaddir)/$(file)")
    
    # Exctract barcode array
    barcode_array_i = graph_output_dict["barcode_array"]
    
    # Compute betti curve for this graph
    for rep = 1:NREPS
        for k in collect(1:MAXDIM)
            
            barcode_i = barcode_array_i[rep,k]
            
            # Filter betti_curve_i based on which half we want
            if half_flag == "first"
                
                # Then we want to set all edges in the second 'half' to the last real edge = thresh edge
                thresh_edge_number = ceil(Int,rho*nEdges)
                barcode_i[barcode_i.> thresh_edge_number].= thresh_edge_number
                
            elseif half_flag == "second"
                # Then we want to set all edges in the first 'half' to the first noise edge = thresh edge + 1
                thresh_edge_number = ceil(Int,rho*nEdges)
                barcode_i[barcode_i.<= thresh_edge_number].= thresh_edge_number+1
            end
                
                
            betti_curve_i = betticurveFromBarcode(barcode_i, nEdges)
            betti_curve_array_all[rep, :, k, i] = betticurveFromBarcode(barcode_i, nEdges)
            
            # Calculate bettiBar, muBar, nuBar
            bettiBar_all[rep, k, i] = bettiBarFromBarcode(barcode_i,nEdges)
            muBar_all[rep, k, i] = muBarFromBarcode(barcode_i,nEdges)
            nuBar_all[rep, k, i] = nuBarFromBarcode(barcode_i,nEdges)
            
        end
    end
    
    println("Finished $(i)")
    
    
end
    
    

Finished 1
Finished 2
Finished 3
Finished 4
Finished 5
Finished 6
Finished 7
Finished 8
Finished 9
Finished 10


In [5]:
# Now to double check we did things right, let's make some Betti Curve plots from our data...

betti_curve_avgs = dropdims(mean(betti_curve_array_all, dims=1), dims=1)
println(size(betti_curve_avgs))

# Make figure
p1a = plot(betti_curve_avgs[:,1,:], layout = 12, size = (1000,800), palette = cgrad(:viridis), title = ["$(model_names[i])" for j=1:1, i=1:nModels])
for d=2:MAXDIM
    plot!(betti_curve_avgs[:,d,:],legend = nothing, xlabel = "Edges added", ylabel="Beta_k")
    
end
# Add vertical line at the noise cutoff
thresh_edge_number = ceil(Int,rho*nEdges)
vline!(transpose([thresh_edge_number for i=1:nModels]), color=:gray)
p1a
savefig("../figures/all_bettis_rho$(thresh)_$(date_string)_$(half_flag).pdf")

(2415, 3, 10)


In [6]:

println(thresh_edge_number)

121


In [7]:
# Save file for jason

matwrite("../processed_data/70nodes/thresh_$(thresh)/bettis_all_rho$(thresh)_$(date_string)_$(half_flag)_forJason.mat", Dict("all_bettis_mat" => betti_curve_array_all, 
        "bettiBar_array" => bettiBar_all, 
        "muBar_array" => muBar_all,
        "nuBar_array" => nuBar_all))